# CNN Testing for Cell Cycle State Classification

### Welcome!

This notebook allows you to take the convolutional neural network (CNN) that you trained in the previous notebook and evaluate its performance on a set of previously unseen single-cell image patches. Follow the step-wise instructions to proceed with testing the network.


### Important Notes:

1. You are using the virtual environment of the [Google Colab](https://colab.research.google.com/notebooks/intro.ipynb "Google Colaboratory"). To be able to test the neural network, you must first **import images not used during training** into the folders to source from. Please follow the instructions after executing the first cell of this notebook.

2. If using Google Colab: This session will 'timeout' if you do not interact with it. It's 90 minutes if you close the browser or 12 hours if you keep the browser open. Additionally, if you close your browser with a code cell is running, if that same cell has not finished, when you reopen the browser it will still be running (the current executing cell keeps running even after browser is closed). Please visit this [StackOverflow](https://stackoverflow.com/questions/54057011/google-colab-session-timeout "Google Colab Session Timeout") discussion for more details.


### Running Instructions:

1. Execute the first cell containing code below, which will install the CellX library & create a local test directory in the environment of the virtual machine. The executed first cell will print ```Building wheel for cellx (setup.py) ... done```. (Note: This virtual environment is different from the one created for the Training notebook, which is why we need to re-install libraries etc.)

2. Click on the ``` 📁``` folder icon located on the left-side dashboard of the Colab notebook. Drag your annotated zip file(s) into the "test" folder. Drag your saved model (the `.h5` file) into the "content" folder - the parent folder of the "test" folder.

3. You can now now run the entire notebook by clicking on ```Runtime``` > ```Run``` in the upper main dashboard. 

---

**Happy testing!**

*Your [CellX](http://lowe.cs.ucl.ac.uk/cellx.html "Lowe Lab @ UCL") team*


### Install the CellX library & create subdirectories in the virtual machine:

In [ ]:
# if using colab, install cellx library and make log and data folders

if 'google.colab' in str(get_ipython()):
    !pip install -q git+git://github.com/quantumjot/cellx.git
    !mkdir test

### Import libraries and CellX toolkit:

In [ ]:
import os
import zipfile
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime
from scipy.special import softmax
from skimage.transform import resize
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
from umap import UMAP

In [ ]:
import tensorflow.keras as K
import tensorflow as tf

In [ ]:
from cellx.core import load_model
from cellx.layers import Encoder2D
from cellx.tools.confusion import plot_confusion_matrix
from cellx.tools.io import read_annotations
from cellx.tools.projection import ManifoldProjection2D

### Define paths & class labels:

In [ ]:
TEST_PATH = "./test"
LABELS = ["Interphase", "Prometaphase", "Metaphase", "Anaphase", "Apoptosis"]

### Import Testing Dataset from the zip files:

In [ ]:
test_images, test_labels, states = read_annotations(TEST_PATH)

### Load the Model

By using the "load_model" function from the CellX library, we can import models without needing to specify the CellX custom layers that had been used to build them.

In [ ]:
model_name = 'model'
model = load_model(f'{model_name}.h5')

In [ ]:
model.summary()

### Normalize the images in the test dataset

In [ ]:
# image normalization function
def normalize_image_array(img):
    img_mean = np.mean(img)
    img_stddev = max(np.std(img), 1.0/np.size(img))
    img = np.subtract(img,img_mean)
    img = np.divide(img,img_stddev)
    # clip to 4 standard deviations
    img = np.clip(img, -4, 4)
    return img

In [ ]:
test_images = [normalize_image_array(image) for image in test_images]
test_images_array = np.array(test_images)[...,np.newaxis] # convert to numpy array for model prediction
test_labels_array = np.array(test_labels)

## Run the Model on the Testing Set

In [ ]:
test_predictions = model.predict(test_images_array)

The 'softmax' function transforms test_predictions into an array of scores for each class for each instance in the testing set. Across classes, the scores sum to one. The class associated with the highest score is the model's 'prediction'.

In [ ]:
test_predictions = softmax(test_predictions,axis=1)

### Predictions on Testing Images

Sample N images out of the testing set to check the model's predictions on them.

In [ ]:
def show_testing_predictions(
    num_examples, # number of testing examples to show
    test_images
):
    plt.figure(figsize=(10,3*(int(num_examples/5)+1)))
    plt.suptitle('Predictions',fontsize=25,x=0.5,y=0.95)
    for image_num in range(min(np.shape(test_images_array)[0],num_examples)-1):
        plt.subplot(int(num_examples/5)+1,5,image_num+1)
        plt.imshow(test_images_array[image_num,:,:,0])
        plt.title('Image {}'.format(image_num+1))
        plt.yticks([])
        plt.xticks([])
        plt.xlabel(LABELS[np.argmax(test_predictions[image_num])])
    plt.show()

In [ ]:
show_testing_predictions(20,test_images)

### Evaluation Metrics

We will next calculate the "precision", "recall" and "F1 score" metrics for each class, as well as the "confusion matrix" for the CNN's performance on the testing set. The three metrics are calculated using the number of "false positive", "true positive" and "false negative" predictions for each class.
- The "precision" of class X is calculated by $$precision(X) = \frac{No.\;of\;true\;positives}{No.\;of\;true\;positives+No.\;of\;false\;positives}$$
- The "recall" of class X is calculated by $$recall(X) = \frac{No.\;of\;true\;positives}{No.\;of\;true\;positives+No.\;of\;false\;negatives}$$
- The "F1 score" of class X is calculated by $$F1(X) = 2*\frac{precision(X)*recall(X)}{precision(X)+recall(X)}$$
<br>

The "confusion matrix" is a table that visually represents the performance of a network on a testing set. The number shown in row A and column B is the number of testing examples of ground-truth class A that have been predicted as belonging to class B by the network.

Reading resource for confusion matrices: https://towardsdatascience.com/understanding-confusion-matrix-a9ad42dcfd62

In [ ]:
loss,accuracy = model.evaluate(test_images_array, test_labels_array)

test_confusion_matrix = confusion_matrix(test_labels,np.argmax(test_predictions,axis=1))
test_confusion_matrix_plot = plot_confusion_matrix(test_confusion_matrix,LABELS)
test_confusion_matrix_plot.show()

print('Testing Accuracy = ',accuracy)
print('Testing Loss = ',loss)

precision,recall,fscore,support = precision_recall_fscore_support(test_labels,np.argmax(test_predictions,axis=1))
print('Testing Precision = ',precision)
print('Testing Recall = ',recall)
print('Testing F1 Score = ',fscore)

### Dimensionality reduction with UMAP

By running the below cell, we see that the model output is an array of 2 dimensions: 
* the 1st dimension corresponds to the number of test images used 
* the 2nd dimension corresponds to the number of possible classes predefined in our model

In [ ]:
test_predictions.shape

We can use UMAP to easily visualise the network's classification performance by embedding the predictions from 5D space (number of classes/features) into a lower 2D space while attempting to keep the data's inherent structure and underlying relationships.

We first define our parameters of choice. In this simple example, we chose to only modify the following ones:
* `n_neighbors` - the number of neighbours determines the size of the local neighbourhood that UMAP should focus on when creating the embedding, low values => emphasis on local structure, high values => emphasis on global structure
* `n_epochs` - the number of epochs determines the number of rounds the UMAP embedding will be optimised for (similar to training a CNN), the higher the number the more accurately the 2D embedding will replicate the original data structure
* `random_state` - UMAP is a stochastic algorithm, so we need to set a random seed to ensure that the results are reproducible across different runs. try eliminating this parameter, you should see slightly different UMAP embeddings from one run to the next

Feel free to adjust the parameters and check how the below image projection changes! You can read up on the most important parameters [here](https://umap-learn.readthedocs.io/en/latest/parameters.html#) or go through the whole list of parameters [here](https://umap-learn.readthedocs.io/en/latest/api.html).

If you're interested in reading about how UMAP works, [see here](https://umap-learn.readthedocs.io/en/latest/basic_usage.html).

In [ ]:
# UMAP parameters
nbs = 5
eps = 50
rnd = 0

We then create a UMAP model with the defined parameters. The full configuration of the UMAP model will be printed out with all the parameter values to be used, including the ones modified above. 

Note:`verbose=True`enables written feedback to the user while UMAP is running.

In [ ]:
mapper = UMAP(n_neighbors=nbs, n_epochs=eps, random_state=rnd, verbose=True)

Fit the UMAP model to the data.

In [ ]:
mapper.fit(test_predictions)

### Image patch projection

By projecting the test images corresponding to the test predictions on top of the UMAP embedding, we can visually assess whether single-cell patches of the same class correctly cluster together in 2D space.

In [ ]:
# convert single-channel test images to rgb three-channel images
print(f"shape of test images: {test_images_array.shape}")
rgb_images = np.concatenate([test_images]*3, axis=-1)
print(f"shape of rgb test images: {rgb_images.shape}")
# normalise image values to 0-1 range (Min-Max scaling) & convert to 8-bit
rgb_images = ((rgb_images-np.min(rgb_images))/(np.ptp(rgb_images)) * 255).astype(np.uint8)

Create the grid of image patches corresponding to the UMAP embedding. (This is basically a 2D histogram where points on a same grid cell are binned and the average of the binned images is projected.)

In [ ]:
projection = ManifoldProjection2D(rgb_images)
img_grid, heatmap, delimiters = projection(mapper.embedding_, components=(0,1))

Create a figure to show the image projection. The last line allows you to save the projection to the Colab "content" folder, but remember to then go on the "..." button next to the file generated in the Files tab (click the Refresh button if you don't see it) in order to download a local copy of the file. Reminder: Files saved during a Colab session will be lost upon closing this session!

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))

im = plt.imshow(img_grid,
                origin="lower",
#                 extent=delimiters, 
                cmap="gray",)

plt.tight_layout()
plt.colorbar()

# (optional) uncomment the below line to save the UMAP image patch projection
# fig.savefig(f"umap_{mapper.n_neighbors}nbs_rnd{mapper.random_state}.png")